# Modeling human behavior
Vaccination and Hygiene are the most important [factors](https://www.youtube.com/watch?v=gxAaO2rsdIs) that influence the growth rate of infection. We will explore the following groups of behaviors (adapted from [this paper](https://arxiv.org/pdf/2101.07926)):
- Daily interactions
- Hygiene
- Vaccination
- Quarantine

In [ ]:
# model vaccination decision based on previous experience
def vaccination_decision(p_n, s, r, phi):
    """ 
    Model vaccination decision based on previous experience.
    parameters:
    - p_n: proportion of population vaccinated in year n
    - s: probability of vaccine success
    - r: vaccine morbidity rate
    - phi: fractional size of pandemic
    return: the percentage of the population that vaccinates in the next time step.
    """
    # input validation
    if not all(0 <= x <= 1 for x in [p_n, s, r, phi]):
        raise ValueError("Input values must be between 0 and 1.")
    s_pn = s * p_n # proportion of population immune to virus in year n
    f = proportion_infected(phi, s_pn)

    term1 = (f * (1 - p_n)) # non-vaccinated individuals who get sick
    term2 = (1 - r) * s * p_n # vaccinated individuals who did not get sick
    term3 = (1 - f) * (1 - r) * (1 - s) * p_n # vaccinated individuals who got sick (unsuccessful)
    
    p_next = term1 + term2 + term3
    
    return min(max(p_next, 0), 1) 


def proportion_infected(phi, s_pn):
    """
    Model the proportion of susceptible individuals that get infected during a pandemic.

    Parameters:
    phi: fractional size of pandemic (0 < phi < 1)
    s_pn: population of susceptible individuals (0 < s_pn < 1)
    """
    # Check input validity
    if not (0 < phi < 1) or not (0 < s_pn < 1):
        raise ValueError("Input values must be between 0 and 1.")

    # model equation
    try:
        f = phi / (1 - s_pn)
        return min(f, 1.0)
    except ZeroDivisionError:
        return 0 # if s_pn is 1, then no one gets infected

### Run simulation
The following sources were used:
- The vaccination rate of NYC is around [50%](https://www.nyc.gov/site/doh/about/press/pr2020/flu-vaccinations-surge-this-season.page)
- Vaccine success of Influenza B is [56.4%](https://www.cidrap.umn.edu/influenza-vaccines/flu-vaccine-estimated-be-21-effective-against-flu-spread-household-members)
- CDC surveillance data shows that 8% of the population gets infected during a flu season

In [ ]:
# run simulation
p_n = .5 # initial vaccination rate
s = .564 # probability of vaccine success 
r = .1 # vaccine morbidity rate
phi = .08 # fractional size of pandemic

print(vaccination_decision(p_n, s, r, phi))